In [1]:
import numpy as np
import os
import pandas as pd

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import pytorch_lightning as pl

import utils as ut
from  data import SentenceDataset, Lang, PredictionDataset
from model import EncoderLayer, DecoderLayer, Model

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'

/home/harsh/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
2023-06-30 19:42:10.175831: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 19:42:13.052271: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-30 19:42:13.052415: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.s

In [2]:
lines = open(f'eng-fra.txt', encoding='utf-8').read().strip().split('\n')
lines = [[s for s in l.split('\t')] for l in lines]
df_lines = pd.DataFrame(columns=['Input', 'Target'], data=lines).drop_duplicates(subset=['Input'])
# [-10:].reset_index(drop=True)
df_lines.head()

,Input,Target
0,Go.,Va !
1,Run!,Cours !
3,Wow!,Ça alors !
4,Fire!,Au feu !
5,Help!,À l'aide !


In [3]:
df_lines['n_words'] = df_lines['Input'].apply(lambda n: len(n.split()))
df_lines = df_lines[df_lines['n_words'] >= 10].reset_index(drop=True)

In [4]:
batch_size = 100
max_sequence_length = 70

input_emb_size = 512
input_num_heads = 8

target_emb_size = 512
target_num_heads = 8

num_heads = 8

In [5]:
input_language = Lang('English')
target_language = Lang('French')
sentencedataset = SentenceDataset(df_lines, input_language, target_language, max_sequence_length)

input language: English, unique words found 6771
target language: French, unique words found 9280


In [6]:
def collate_fn(batch):
    input_batch, target_batch = [], []
    for input_sentence, target_sentence in batch:
        input_batch.append(input_sentence)
        target_batch.append(target_sentence)
    input_batch = pad_sequence(input_batch, batch_first=True,
                               padding_value=input_language.word2index['PAD'])
    target_batch = pad_sequence(target_batch, batch_first=True,
                                padding_value=target_language.word2index['PAD'])
    return input_batch, target_batch

In [7]:
sentence_dataloader = torch.utils.data.DataLoader(sentencedataset, 
                                                  batch_size=batch_size,
                                                  collate_fn=collate_fn)

In [8]:
# next(iter(sentence_dataloader))

In [10]:
encoderlayer = EncoderLayer(input_embeddings_size=input_emb_size, input_vocab_size=input_language.n_words,
                            num_heads=input_num_heads, max_sequence_length=max_sequence_length)
decoderlayer = DecoderLayer(target_embeddings_size=target_emb_size, target_vocab_size=target_language.n_words,
                            num_heads=target_num_heads, max_sequence_length=max_sequence_length)
model = Model(encoderlayer, decoderlayer, max_sequence_length, input_language, target_language)

In [11]:
trainer = pl.Trainer(accelerator='gpu', devices=[0], max_epochs=30)
trainer.fit(model, train_dataloaders=sentence_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | encoder   | EncoderLayer     | 4.8 M 
1 | decoder   | DecoderLayer     | 7.1 M 
2 | linear    | Linear           | 4.8 M 
3 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
16.7 M    Trainable params
0         Non-trainable params
16.7 M    Total params
66.632    Total estimated model params size (MB)
/home/harsh/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve p

Training: 0it [00:00, ?it/s]

In [ ]:
predictiondataset = PredictionDataset(sentences=df_lines, input_language=input_language, 
                                      target_language=target_language, max_sequence_length=max_sequence_length)
prediction_dataloader = torch.utils.data.DataLoader(predictiondataset, batch_size=1)
pred_batch = next(iter(prediction_dataloader))

In [ ]:
def sentence_from_index(lang, indices):
    sentence = ''
    for index in indices:
        if lang.index2word[index]!='PAD':
            sentence += f'{lang.index2word[index]} '
    return sentence

In [ ]:
model = model.to(device='cpu')

In [ ]:
iterator = iter(prediction_dataloader)

In [ ]:
pred_batch = next(iterator)
preds = model.predict_step(pred_batch)

In [27]:
src = pred_batch.detach().cpu().numpy()
preds = preds.detach().cpu().numpy()
src.shape, preds.shape

((1, 11), (1, 15))

In [28]:
inference_index = 0
src_sentence = sentence_from_index(input_language, src[0])
preds_sentence = sentence_from_index(target_language, preds[0])
src_sentence, preds_sentence

('it is a lot of fun to drive a car EOS ',
 'SOS c est tres amusant de conduire une voiture voiture voiture en est rouler EOS ')